In [1]:
import os, pickle, logging, pickle, joblib, sys, warnings
warnings.simplefilter('ignore')
from scipy import stats
import numpy as np
import pandas as pd

import catboost as cb
import lightgbm as lgb

from sklearn import ensemble, metrics, pipeline, preprocessing, impute, model_selection
from scipy.stats import pearsonr, spearmanr

ML_RAW_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/rawData/"
FS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/fs_PL/"
TEST_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/tesRes/"

In [2]:
feat_col = []
with open(os.path.join(FS_PATH, "lgb_rfa_cv_pl_rmse")) as f:
    for line in f:
        feat_col.append(line.strip())

raw_data = pd.read_table(os.path.join(ML_RAW_PATH, "raw_data_PL.txt"), sep="\t", index_col=0)

target_col = ['HZ-PL']
raw_data = raw_data.dropna(subset=target_col)

#cat_col_names_raw = ["heading_stage_d", "leaf_blast_rep1", "leaf_blast_rep2", "leaf_blast_average", "GR_D3", "GR_D2", "GR_D1"]
#cat_col_names_raw = ["feat_{}".format(x) for x in cat_col_names_raw]
#cat_col_names = [i for i in feat_col if i in cat_col_names_raw]
#num_col_names = [i for i in feat_col if (i.startswith("feat")) and (i not in cat_col_names)]
#cat_col_num = []
#for item in cat_col_names:
    #cat_col_num.append(feat_col.index(item))

#for col in cat_col_names:
    #raw_data[col] = raw_data[col].astype("str")

kf = model_selection.KFold(n_splits=5, shuffle=True,  random_state=0)
y_test_final, y_pred_final = [], []
for i, (train_index, test_index) in enumerate(kf.split(raw_data)):
    data_train = raw_data.iloc[train_index].copy()
    data_test = raw_data.iloc[test_index].copy()
    
    scale_tool = preprocessing.StandardScaler()
    scale_tool.fit(data_train.loc[:, feat_col])
    data_train.loc[:, feat_col] = scale_tool.transform(data_train.loc[:, feat_col])
    data_test.loc[:, feat_col] = scale_tool.transform(data_test.loc[:, feat_col])

    train_sel = data_train.sample(frac=0.8, random_state=0)
    val_sel = data_train.drop(train_sel.index).copy()

    X_train = train_sel[feat_col].copy()
    y_train = train_sel[target_col].values.ravel()

    X_val = val_sel[feat_col].copy()
    y_val = val_sel[target_col].values.ravel()

    X_test = data_test[feat_col].copy()
    y_test = data_test[target_col].values.ravel()

    # Initialize CatBoostClassifier
    clf_model = lgb.LGBMRegressor(boosting_type="gbdt", n_estimators=1000, random_state=0, n_jobs=4, num_leaves=5)

    clf_model.fit(X_train, y_train, 
                  eval_set=[(X_train, y_train), (X_val, y_val)],
                  callbacks=[lgb.log_evaluation(period=10), 
                             lgb.early_stopping(stopping_rounds=10)])

    y_pred = clf_model.predict(X_test)

    y_test_final.append(y_test)
    y_pred_final.append(y_pred)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 228
[LightGBM] [Info] Number of data points in the train set: 192, number of used features: 24
[LightGBM] [Info] Start training from score 42.910401
Training until validation scores don't improve for 10 rounds
[10]	training's l2: 102.482	valid_1's l2: 122.35
[20]	training's l2: 66.9743	valid_1's l2: 89.4419
[30]	training's l2: 55.5334	valid_1's l2: 82.5561
[40]	training's l2: 49.5444	valid_1's l2: 82.0045
[50]	training's l2: 45.6197	valid_1's l2: 79.807
[60]	training's l2: 42.4523	valid_1's l2: 78.6026
[70]	training's l2: 39.7582	valid_1's l2: 77.3776
[80]	training's l2: 36.7056	valid_1's l2: 77.0675
[90]	training's l2: 34.0829	valid_1's l2: 77.5525
Early stopping, best iteration is:
[82]	training's l2: 36.2181	valid_1's l2: 76.8452

In [5]:
res_df = pd.DataFrame(columns=["Model", "Times", "Score", "Type"])
for i in range(len(y_test_final)):
    ## Get score
    #score_pear = pearsonr(y_test_final[i], y_pred_final[i])[0]
    score_spear = spearmanr(y_test_final[i], y_pred_final[i])[0]
    #score_rmse = metrics.mean_squared_error(y_test_final[i], y_pred_final[i], squared=False)
    score_rmse = metrics.root_mean_squared_error(y_test_final[i], y_pred_final[i])
    score_nrmse = score_rmse / np.std(y_test_final[i])

    #res_df.loc[len(res_df)] = ["LB_RFA", i+1, score_pear, "R"]
    res_df.loc[len(res_df)] = ["LB_RFA", i+1, score_spear, "R"]
    res_df.loc[len(res_df)] = ["LB_RFA", i+1, score_rmse, "RMSE"]
    res_df.loc[len(res_df)] = ["LB_RFA", i+1, score_nrmse, "NRMSE"]

with open(os.path.join(TEST_RES_PATH, "PL_LB_RFA.pickle"), "wb") as out_f:
    pickle.dump(res_df, out_f)

In [6]:
res_df.groupby(["Type"])["Score"].mean()

Type
NRMSE    0.478180
R        0.862490
RMSE     7.805086
Name: Score, dtype: float64

In [2]:
with open(os.path.join(TEST_RES_PATH, "PL_LB_RFA.pickle"), "rb") as f:
    res_df = pickle.load(f)

res_df

,Model,Times,Score,Type
0,LB_RFA,1,0.904466,R
1,LB_RFA,1,6.707625,RMSE
2,LB_RFA,1,0.403047,NRMSE
3,LB_RFA,2,0.869893,R
4,LB_RFA,2,8.409943,RMSE
5,LB_RFA,2,0.471744,NRMSE
6,LB_RFA,3,0.906153,R
7,LB_RFA,3,7.686936,RMSE
8,LB_RFA,3,0.472184,NRMSE
9,LB_RFA,4,0.804612,R


In [3]:
res_df.groupby(["Type"])["Score"].mean()

Type
NRMSE    0.473975
R        0.861682
RMSE     7.663504
Name: Score, dtype: float64